In an effort to start learning some patterns in ML, I decided to go with small steps and reimplement some patterns in OCaml. Assumming the concept of `list` doesn't exist yet we will reimplement a similar concept known as `sequence`:

In [56]:
type 'a seq =
  | End
  | Next of 'a * 'a seq
;;

type 'a seq = End | Next of 'a * 'a seq


We can define a sequence of zero elements like this:

In [58]:
let no_elements = End

val no_elements : 'a seq = End


Or the count from 1 to 3 like this:

In [59]:
let sample = Next(1, Next(2, Next(3, End)))

val sample : int seq = Next (1, Next (2, Next (3, End)))


### Implement basic operations

Length, Head, Tail, Cons

In [60]:
(* length operator *)
let length_seq x =
  let rec length' acc = function
    | End -> acc
    | Next (_, t) -> length' (acc + 1) t
  in
  length' 0 x
;;

length_seq sample;;

(* head operator *)
let head_seq = function
  | End -> raise (Invalid_argument "empty sequence")
  | Next (h, _) -> h
;;

head_seq sample;;

(* tail operator *)
let tail_seq = function
  | End -> raise (Invalid_argument "empty sequence")
  | Next (_, t) -> t
;;

tail_seq sample;;

(* cons operator *)
let cons_seq x l = Next(x, l) ;;
cons_seq 0 sample ;;

val length_seq : 'a seq -> int = <fun>


- : int = 3


val head_seq : 'a seq -> 'a = <fun>


- : int = 1


val tail_seq : 'a seq -> 'a seq = <fun>


- : int seq = Next (2, Next (3, End))


val cons_seq : 'a -> 'a seq -> 'a seq = <fun>


- : int seq = Next (0, Next (1, Next (2, Next (3, End))))


Now we can start implementing even more complex operations, like `nth`, `rev`, `append`

In [61]:
(* nth operator *)
let rec nth_seq n l = 
  if n < 0 then raise (Invalid_argument "nth") else
  match l with
    | End -> raise (Failure "nth")
    | Next (x, tl) -> if n = 0 then x else nth_seq (n - 1) tl
;;

nth_seq 0 sample ;;
nth_seq 2 sample ;;

(* rev operator *)
let rev_seq l =
  let rec rev' acc = function
    | End -> acc
    | Next (x, tl) -> rev' (cons_seq x acc) tl
  in
  rev' End l
;;

rev_seq sample ;;

(* append operator, this function is not tail recursive *)
(* the original in OCaml neither *)
let rec append_seq a b =
  match a with
   | End -> b
   | Next (x, tl) -> Next(x, ((append_seq [@tailcall]) tl b))
;;

append_seq sample (Next(4, Next(5, Next(6, End))))

(* We can make a recursive append *)
let append_seq' a b =
  let rec append' acc = function
    | End -> b
    | Next (x, t) -> append' (Next(x, acc)) t
  in
  append' End (rev_seq a)
;;

append_seq sample (Next(4, Next(5, Next(6, End)))) ;;

let (+:) a b = cons_seq a b ;;
0 +: sample

val nth_seq : int -> 'a seq -> 'a = <fun>


- : int = 1


- : int = 3


val rev_seq : 'a seq -> 'a seq = <fun>


- : int seq = Next (3, Next (2, Next (1, End)))


...

- : int seq = Next (1, Next (2, Next (3, Next (4, Next (5, Next (6, End))))))


val append_seq' : 'a seq -> 'b -> 'b = <fun>


- : int seq = Next (1, Next (2, Next (3, Next (4, Next (5, Next (6, End))))))


val ( +: ) : 'a -> 'a seq -> 'a seq = <fun>


- : int seq = Next (0, Next (1, Next (2, Next (3, End))))


Now we can start playing with functions and lists. What about `init`, `map` and, more importantly, `fold_right` and `fold_left`?

In [62]:
(* init operator *)
let init_seq n f =
  let rec init' n' acc =
    if n' = 0 then acc else init' (n' - 1) (cons_seq (f n') acc)
  in
  init' n End
;;

init_seq 3 (fun x -> x) ;;

(* map operator, not tail recursive *)
let rec map_seq f = function
  | Next (x, t) -> Next ((f x), ((map_seq [@tailcall]) f t))
  | _ -> End
;;

map_seq (fun x -> x * 2) sample ;;

(* map operator, now tail recursive *)
let map_seq ~f l =
  let rec map_seq' acc = function
    | End -> acc
    | Next (x, t) -> map_seq' (Next((f x), acc)) t
  in
  rev_seq (map_seq' End l)
;;

map_seq ~f:(fun x -> x * 2) sample

val init_seq : int -> (int -> 'a) -> 'a seq = <fun>


- : int seq = Next (1, Next (2, Next (3, End)))


val map_seq : ('a -> 'b) -> 'a seq -> 'b seq = <fun>


File "[62]", line 13, characters 32-59:
Warning 51: expected tailcall


- : int seq = Next (2, Next (4, Next (6, End)))


val map_seq : f:('a -> 'b) -> 'a seq -> 'b seq = <fun>


- : int seq = Next (2, Next (4, Next (6, End)))


Now it is time for the `fold` functions. The fold functions is just a nicer way to say "aggregate" functions.

In [105]:
let rec fold_left_seq ~f ~init = function
  | End -> init
  | Next (x, tl) -> (fold_left_seq [@tailcall]) ~f ~init:(f init x) tl
;;

fold_left_seq ~f:(fun a b -> a + b) ~init:0 sample

val fold_left_seq : f:('a -> 'b -> 'a) -> init:'a -> 'b seq -> 'a = <fun>


- : int = 6


the contrapart of `fold_left_seq` is `fold_right_seq`, we can implement it in two ways, a non-tail recursive and a tail recursive version, let's see both:

In [143]:
(* Non tail recursive fold_right *)
let rec fold_right_seq ~f l ~init =
  match l with
  | End -> init
  | Next (x, tl) -> f x ((fold_right_seq [@tailcall]) ~f tl ~init)
;;

(* tail recursive fold_right *)
(* Notice we have to _reverse_ the list and change the signature of ~f *)
let rec fold_right_seq ~f l ~init = 
  fold_left_seq ~f:(fun a b -> f b a) (rev_seq l) ~init ;;

File "[143]", line 5, characters 24-66:
Warning 51: expected tailcall


val fold_right_seq : f:('a -> 'b -> 'b) -> 'a seq -> init:'b -> 'b = <fun>


val fold_right_seq : f:('a -> 'b -> 'b) -> 'a seq -> init:'b -> 'b = <fun>


When to use which? well, it depends, in _theory_ the non-tail recursive is more efficient but of course it cannot work in long sequences, being $O(n)$ while the tail recursive is $O(n^2)$. **Remember**, `cons_seq` is $O(1)$, while `rev_seq` is $O(n)$ (the same as `fold_left_seq`) and when using _reverse inner accumulator loop_ we get $O(2n)$ which is basically $O(n)$

The difference between both is about memory and performance. For _large sequences_ our tail recursive version is the only option while for short sequences is prefered to use the non tail recursive.

We can implement other basic sequence functions based in `fold_left`, for example, the always important function `rev`

In [125]:
let rev_seq' = 
  fold_left_seq ~f:(fun l' x -> cons_seq x l') ~init:End 
;;

rev_seq' sample

val rev_seq' : '_weak8 seq -> '_weak8 seq = <fun>


- : int seq = Next (3, Next (2, Next (1, End)))


Notice how we are _currying_ the function definition, because _why not?_.

More complex functions can be represented with fold as well, for example, `map`

In [144]:
let map_seq' ~f =
  fold_right_seq ~f:(fun elem l -> cons_seq (f elem) l) ~init:End
;;
sample ;;
map_seq' ~f:(fun x -> x * 2) sample

val map_seq' : f:('a -> 'b) -> 'a seq -> 'b seq = <fun>


- : int seq = Next (1, Next (2, Next (3, End)))


- : int seq = Next (2, Next (4, Next (6, End)))
